<a href="https://colab.research.google.com/github/Farivini/Extracting-Statistics-in-PySpark/blob/main/Analises_descritivas_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixar o dataset do Kaggle
!kaggle datasets download -d camnugent/sandp500
!unzip sandp500.zip

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length, col, countDistinct



In [ ]:
spark= SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.csv("/content/all_stocks_5yr.csv", header=True)

In [ ]:
df.show()

### Descrição sobre o dataset

In [ ]:
df.describe().show()

### Registro sobre a apple

In [ ]:
aapl = df.filter(df.Name == "AAPL")
aapl_cont = aapl.count()
print(aapl_cont)

### Empresas distintas

In [ ]:
empresas_distintas = df.select("Name").distinct().count()
print(empresas_distintas)

### Frequencia com que o preço de uma ação no fechamento é maior que o preço na abertura

In [ ]:
# filtro para fechamentos maior
fechamento_maior_abertura = df.filter(df.close > df.open)
contagem_maior_abertura = fechamento_maior_abertura.count()
total = df.count()
frequencia_maior_abertura = contagem_maior_abertura / total
print(frequencia_maior_abertura*100)

### Qual o maior valor das ações da Apple (AAPL) na Historia ?

In [ ]:
aapl_max = aapl.agg({"High": "max"}).collect()[0][0]
print(aapl_max)

In [ ]:
apple_maximo = df.filter(df.Name == "AAPL")
apple_maximo.describe().show()

### Qual ação tem a maior volatilidade?

## Vamos calcular o desvio padrão e depois ordenar e pegar o nome e a maior volatilidade

---



In [ ]:
from pyspark.sql.functions import stddev, col

# Calcular o desvio padrão do preço de fechamento para cada ação
volatilidade = df.groupBy('Name').agg(stddev('Close').alias('Volatility'))
# Ordenar as ações pela maior volatilidade
ordena_volatilidade = volatilidade.orderBy(col('Volatility').desc()).first()

# Mostrar a ação com a maior volatilidade e o valor da volatilidade
print(f"A ação com a maior volatilidade é {ordena_volatilidade['Name']} com uma volatilidade de {ordena_volatilidade['Volatility']:.2f}")


### Qual dia com maior volume total de negociação da bolsa

In [ ]:
maior_dia = df.groupBy('date').agg({'volume': 'sum'}).orderBy(col('sum(volume)').desc()).first()
maior_dia

### Qual a ação mais negociada da bolsa , em volume

In [ ]:
maior_acao = df.groupBy('Name').agg({'volume': 'sum'}).orderBy(col('sum(volume)').desc()).first()
maior_acao

#### Quantas ações começam com a letra A

In [ ]:
start_a = df.filter(df.Name.startswith('A')).distinct().count()
start_a

### Com qual frequencia o preço mais alto do dia da ação tambem é o preço de fechamento?

In [ ]:
freq_maior_preco_fechamento = df.filter(df.high == df.close).count()
print(f"Quantidade de fechamentos que é igual ao fechamento:{freq_maior_preco_fechamento} " )

percentual_maior_preco_fechamento = (freq_maior_preco_fechamento / df.count()) * 100
print(f"Percentual de fechamentos que é igual ao fechamento:{percentual_maior_preco_fechamento:.2f}%")

### Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta e seu grafico.

In [ ]:
maior_dia_apple = df.filter(df.Name == 'AAPL').withColumn('Diferenca', df.close - df.open).orderBy(col('Diferenca').desc()).first()
maior_dia_apple

In [ ]:
grafico_maior_dia_apple = df.filter(df.Name == 'AAPL').withColumn('Diferenca', df.close - df.open).orderBy(col('Diferenca').desc())

grafico_de_linha = grafico_maior_dia_apple.toPandas()
grafico_de_dispeção = grafico_de_linha.plot(x='date', y='Diferenca', kind='line', title='Diferença entre o fechamento e abertura', figsize=(10, 6))

### Qual a media de volume da APPLE?

In [ ]:
media_apple = df.filter(df.Name == 'AAPL').agg({'volume': 'avg'}).collect()[0][0]
media_apple

##### Quais ações tem 1,2,3,4,e 5 caracteres em seu nome respectivamente?

In [ ]:




# Criar uma nova coluna com o comprimento dos nomes das ações
df_with_length = df.withColumn('name_length', length(col('Name')))

# Agrupar pelo comprimento dos nomes e contar o número de nomes distintos para cada comprimento
contar_nomes_distintos = df_with_length.groupBy('name_length').agg(countDistinct('Name').alias('distinct_name_count'))

# Mostrar o resultado
contar_nomes_distintos.show()

grafico_histograma = contar_nomes_distintos.toPandas()
grafico_histograma.plot(x='name_length', y='distinct_name_count', kind='bar', title='Contagem de Nomes Distintos por Comprimento', figsize=(10, 6))

### Qual a ação da bolsa menos negociada da bolsa, em volume de transações?

In [ ]:
acao_menos_negociada = df.groupBy('Name').agg({'volume': 'sum'}).orderBy(col('sum(volume)').asc()).first()
acao_menos_negociada